<a href="https://colab.research.google.com/github/rebelahsan/Mo-Ahsan-Ahmad/blob/main/5P77_SPN_Project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mo Ahsan Ahmad
### COSC 5P77, SPN Project

# SPN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.init as init

# Loading MNIST dataset
norm= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
X_train = datasets.MNIST(root="./data", train=True, transform=norm, download=True)
X_test = datasets.MNIST(root="./data", train=False, transform=norm, download=True)

#DataLoader
n_batch = 64
trn_ldr = DataLoader(X_train, batch_size=n_batch, shuffle=True) #{60000}/{64} = 937.5=938
tst_ldr = DataLoader(X_test, batch_size=n_batch, shuffle=False) # similarly {10000}/{64} =157

# Initializing weights using xavier
def weights_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        init.constant_(m.bias, 0.0)

class SimpleSPN(nn.Module):
    def __init__(self, i_size, sn_size, pn_size, classes):
        super(SimpleSPN, self).__init__()

        #lists to hold sum and product layers
        self.s_layers = nn.ModuleList()
        self.p_layers = nn.ModuleList()

        curr_i_size = i_size

        # Define sum and product layers
        for s_size, p_size in zip(sn_size, pn_size):
            self.s_layers.append(nn.Sequential(nn.Linear(curr_i_size, s_size),nn.ReLU()))
            self.p_layers.append(nn.Linear(s_size, p_size))
            curr_i_size = p_size
        self.op_layer = nn.Linear(curr_i_size, classes)

    def forward(self, x):
        # Process the input through sum and product layers
        for s_layer, p_layer in zip(self.s_layers, self.p_layers):
            x = s_layer(x)
            x = p_layer(x)

        # Final op layer
        op = self.op_layer(x)
        return op

i_size = 28 * 28
sn_size = [128, 64]
pn_size = [64, 32]
classes = 10

#SPN model
spn_model = SimpleSPN(i_size, sn_size, pn_size, classes)


#Cross Entropy Loss)
crit = nn.CrossEntropyLoss()

#optimizers
optimizers = {
    "Adam": optim.Adam(spn_model.parameters(), lr=0.001),
    "RMSprop": optim.RMSprop(spn_model.parameters(), lr=0.001),
    "Adamax": optim.Adamax(spn_model.parameters(), lr=0.001),
}

def train_model(optimizer, model, crit, data_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        for btch_id, (data, target) in enumerate(data_loader):
            data = data.view(-1, i_size)
            optimizer.zero_grad()
            op = model(data)
            loss = crit(op, target)
            loss.backward()
            optimizer.step()

            if btch_id % 100 == 0:
                f"Batch {btch_id}, Loss: {loss.item():.4f}"
        #print()

def test_model(model, crit, tst_ldr):
    model.eval()
    ini_loss = 0
    crct = 0
    total = 0

    with torch.no_grad():
        for data, target in tst_ldr:
            data = data.view(-1, i_size)
            op = model(data)
            loss = crit(op, target)
            ini_loss += loss.item()

            predicted = op.argmax(dim=1)
            crct += (predicted == target).sum().item()
            total += target.size(0)

    print(f"Avg Test Loss: {ini_loss / len(tst_ldr):.4f}")
    print(f"Accuracy: {100. * crct / total:.2f}")

# Train/test the model with optimizers
for optimizer_name, optimizer in optimizers.items():
    print(f"Optimizer: {optimizer_name}")
    #setup model parameters
    spn_model.apply(weights_init)
    train_model(optimizer, spn_model, crit, trn_ldr)
    test_model(spn_model, crit, tst_ldr)
    print()


Optimizer: Adam
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.1014
Accuracy: 97.23

Optimizer: RMSprop
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.1041
Accuracy: 96.93

Optimizer: Adamax
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.0827
Accuracy: 97.52



# RNN

In [3]:
#RNN model
class RNN(nn.Module):
    def __init__(self, i_size, hid_size, n_layers, classes):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(i_size, hid_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hid_size, classes)
        self.i_size = i_size
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
            init.constant_(m.bias, 0.0)

    def forward(self, x):
        # Flatten the input & Forward pass through RNN
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.i_size)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

i_size = 28 * 28
hid_size = 128
n_layers = 2  #recurrent layers
classes = 10
rnn_model = RNN(i_size, hid_size, n_layers, classes)

# Cross Entropy Loss
crit = nn.CrossEntropyLoss()

# optimizers
optimizers = {
    "Adam": optim.Adam(rnn_model.parameters(), lr=0.001),
    "RMSprop": optim.RMSprop(rnn_model.parameters(), lr=0.001),
    "Adamax": optim.Adamax(rnn_model.parameters(), lr=0.001),
}

def train_model(optimizer, model, crit, data_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for batch_idx, (data, target) in enumerate(data_loader):
            optimizer.zero_grad()
            op = model(data)
            loss = crit(op, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                f"Batch {batch_idx}, Loss: {loss.item():.4f}"

def test_model(model, crit, test_loader):
    model.eval()
    ini_loss = 0
    crct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            op = model(data)
            loss = crit(op, target)
            ini_loss += loss.item()
            _, predicted = torch.max(op, 1)
            crct += (predicted == target).sum().item()
            total += target.size(0)

    print(f"Avg Test Loss: {ini_loss / len(test_loader):.4f}")
    print(f"Accuracy: {100. * crct / total:.2f}")

# Train and test the model using different optimizers
for optimizer_name, optimizer in optimizers.items():
    print(f"Optimizer: {optimizer_name}")
    train_model(optimizer, rnn_model, crit, trn_ldr)
    test_model(rnn_model, crit, tst_ldr)
    print()


Optimizer: Adam
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.0996
Accuracy: 97.05

Optimizer: RMSprop
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.0960
Accuracy: 97.19

Optimizer: Adamax
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Avg Test Loss: 0.0819
Accuracy: 97.79

